Basic setup and data loading

In [ ]:
%pip install llama-index-vector-stores-weaviate
%pip install llama-index

In [ ]:
pip install nest_asyncio

In [18]:
pip install matplotlib

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.2/162.2 kB 4.1 MB/s eta 0:00:00
  Using cached pyparsing-3.1.2-py3-none-any.whl.metadata (5.1 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 31.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 21.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 kB 9.2 MB/s eta 0:00:00
Using cached pyparsing-3.1.2-py3-none-any.whl (103 kB)
Note: you may need to restart the kernel to use updated packages.


In [3]:
# setup Arize Phoenix for logging/observability
import phoenix as px
import llama_index.core
px.launch_app()
llama_index.core.set_global_handler("arize_phoenix")

No active session to close


E0628 10:04:39.456699000 13218328576 chttp2_server.cc:1097]            UNKNOWN:No address added out of total 1 resolved for '[::]:4317' {created_time:"2024-06-28T10:04:39.456642-07:00", children:[UNKNOWN:Failed to add any wildcard listeners {created_time:"2024-06-28T10:04:39.456634-07:00", children:[UNKNOWN:Unable to configure socket {fd:90, created_time:"2024-06-28T10:04:39.456564-07:00", children:[UNKNOWN:Address already in use {syscall:"bind", os_error:"Address already in use", errno:48, created_time:"2024-06-28T10:04:39.456546-07:00"}]}, UNKNOWN:Unable to configure socket {fd:90, created_time:"2024-06-28T10:04:39.45663-07:00", children:[UNKNOWN:Address already in use {created_time:"2024-06-28T10:04:39.456621-07:00", errno:48, os_error:"Address already in use", syscall:"bind"}]}]}]}
ERROR:    Traceback (most recent call last):
  File "/Users/haiyangliu/Workspace/gov_opportunity_new/gov_env/lib/python3.10/site-packages/starlette/routing.py", line 732, in lifespan
    async with self.

In [4]:
import os
import json 
import openai

with open('openai.secret.json', 'r') as file:
    secret = json.load(file)
    os.environ["OPENAI_API_KEY"] = secret['secret']
    openai.api_key = os.environ["OPENAI_API_KEY"]

In [5]:
import weaviate
from llama_index.vector_stores.weaviate import WeaviateVectorStore
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings


client = weaviate.connect_to_local(
    headers={"X-OpenAI-Api-key": os.getenv("OPENAI_API_KEY")}
)

Settings.embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-small-en-v1.5"
)

/Users/haiyangliu/Workspace/gov_opportunity_new/gov_env/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [7]:
from llama_index.vector_stores.weaviate import WeaviateVectorStore
from llama_index.core import VectorStoreIndex, StorageContext

vector_store = WeaviateVectorStore(
    weaviate_client=client, index_name='Gov_projects'
)
storage_context = StorageContext.from_defaults(vector_store=vector_store)


In [8]:
import nest_asyncio
nest_asyncio.apply()

In [12]:
from llama_index.core import SimpleDirectoryReader
import json
from dateutil import parser
from llama_index.core.async_utils import run_jobs
from llama_index.core import SummaryIndex, Document
from llama_index.llms.openai import OpenAI
from llama_index.core.extractors import BaseExtractor
from llama_index.core.node_parser import JSONNodeParser



documents = SimpleDirectoryReader('/Users/haiyangliu/Workspace/gov_opportunity_new/content/opportunities/projects').load_data()
def augment_with_metadata(document: Document):
    doc_object = json.loads(document.get_text())
    new_metadata = {
    "project_id": doc_object['ProjectID'],
    'department': doc_object['Department/Ind.Agency'],
    'naics_code': doc_object['NaicsCode'],
    'psc_code': doc_object['ProductServiceCode'],
    'sub_department': doc_object['Sub-Tier'],
    'set_aside': doc_object['SetASide'],
    'set_aside_code': doc_object['SetASideCode'],
    'type': doc_object['Type'],
    'place_of_performance_country': doc_object['PlaceOfPerformanceCountry'],
    'place_of_performance_city': doc_object['PlaceOfPerformanceCity'],
    'place_of_performance_state': doc_object['PlaceOfPerformanceState'],
    }
    if (doc_object['ResponseDueDate']):
        due_date = parser.parse(doc_object['ResponseDueDate'])
        new_metadata["due_date_year"] = due_date.year
        new_metadata["due_date_month"] = due_date.month
        new_metadata["due_date_day"] = due_date.day

    if (doc_object['PostedDate']):
        posted_date = parser.parse(doc_object['PostedDate'])
        new_metadata["posted_date_year"] = posted_date.year
        new_metadata["posted_date_month"] = posted_date.month
        new_metadata["posted_date_day"] = posted_date.day
    document.metadata.update(new_metadata)
    document.excluded_llm_metadata_keys = ['file_path', 'file_type', 'file_size', 'creation_date', 'last_modified_date']
    document.excluded_embed_metadata_keys = ['file_path', 'file_type', 'file_size', 'creation_date', 'last_modified_date']
    document.metadata_seperator="::",
    document.metadata_template="{key}=>{value}",
    document.text_template="Metadata: {metadata_str}\n-----\nContent: {content}",

for doc in documents:
    augment_with_metadata(doc)
    # print(doc.metadata)

{'file_path': '/Users/haiyangliu/Workspace/gov_opportunity_new/content/opportunities/projects/project_41c2be0ed45342a687cad02d4d20b1cd.json', 'file_name': 'project_41c2be0ed45342a687cad02d4d20b1cd.json', 'file_type': 'application/json', 'file_size': 6146, 'creation_date': '2024-06-27', 'last_modified_date': '2024-06-27', 'project_id': '41c2be0ed45342a687cad02d4d20b1cd', 'department': 'Department of Defense', 'naics_code': '236220', 'psc_code': 'Y1JZ', 'sub_department': 'Dept of the Navy', 'set_aside': '', 'set_aside_code': '', 'type': 'Solicitation', 'place_of_performance_country': '', 'place_of_performance_city': '', 'place_of_performance_state': '', 'due_date_year': 2024, 'due_date_month': 7, 'due_date_day': 9, 'posted_date_year': 2024, 'posted_date_month': 6, 'posted_date_day': 18}
{'file_path': '/Users/haiyangliu/Workspace/gov_opportunity_new/content/opportunities/projects/project_814bba4ad7054ecf8e06dd9185c6a7ce.json', 'file_name': 'project_814bba4ad7054ecf8e06dd9185c6a7ce.json', 

In [32]:
from llama_index.core.node_parser import JSONNodeParser
from llama_index.core.ingestion import IngestionPipeline
from llama_index.core.extractors import SummaryExtractor, TitleExtractor
from llama_index.core.node_parser import TokenTextSplitter

json_parser = JSONNodeParser()

transformations=[
    JSONNodeParser(),
    # TitleExtractor(),
    SummaryExtractor(summaries=["prev", "self"], llm=OpenAI(model="gpt-3.5-turbo")),
    TokenTextSplitter(),
    HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5"),
]

pipeline = IngestionPipeline(
    transformations=transformations,
)
nodes = pipeline.run(documents=documents)

for node in nodes:
    print('*********')
    print(node.ref_doc_id)
    print(node.metadata)
    print(node.get_content())

/Users/haiyangliu/Workspace/gov_opportunity_new/gov_env/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


hybrid keyword search with vector search

In [33]:
from llama_index.core import VectorStoreIndex

basic_index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context, transformations=transformations
)

100%|██████████| 3/3 [00:03<00:00,  1.20s/it]


In [19]:
from llama_index.core.response.notebook_utils import display_response


query_engine = basic_index.as_query_engine(similarity_top_k=2)
response = query_engine.query("what are the plumbing related projects")

display_response(response)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
WARNI [matplotlib.font_manag

**`Final Response:`** REPLACE PLUMBING AND FIRE PROTECTION SYSTEMS (B150), TIN CITY, ALASKA (TIN006)

In [34]:

# Favor vector search
query_engine = basic_index.as_query_engine(
    vector_store_query_mode="hybrid", similarity_top_k=2
)
response = query_engine.query(
    "what are the ids of the projects posted from department of defence",
)
display_response(response)

**`Final Response:`** 41c2be0ed45342a687cad02d4d20b1cd, 814bba4ad7054ecf8e06dd9185c6a7ce

In [35]:

# Favor BM25 (key word search)
query_engine = basic_index.as_query_engine(
    vector_store_query_mode="hybrid", similarity_top_k=2, alpha=0.0
)
response = query_engine.query(
    "what are the ids of the projects posted from department of defence",
)
display_response(response)

**`Final Response:`** 41c2be0ed45342a687cad02d4d20b1cd, 814bba4ad7054ecf8e06dd9185c6a7ce

In [37]:

# Favor BM25 (key word search)
query_engine = basic_index.as_query_engine(
    vector_store_query_mode="hybrid", similarity_top_k=2, alpha=0.0
)
response = query_engine.query(
    "give me the summary of the project 41c2be0ed45342a687cad02d4d20b1cd, max 3 sentense",
)
display_response(response)

**`Final Response:`** The project 41c2be0ed45342a687cad02d4d20b1cd involves a two-phase design-build multiple award construction contract primarily for U.S. military construction projects in Guam and other areas under the cognizance of NAVFAC Pacific. It is a Department of Defense project with a focus on constructing various military facilities such as barracks, administrative buildings, and communication facilities. The procurement process includes evaluation factors like technical approach, experience, performance confidence assessment, safety, and small business utilization.

In [38]:

# Favor BM25 (key word search)
query_engine = basic_index.as_query_engine(
    vector_store_query_mode="hybrid", similarity_top_k=2, alpha=0.6
)
response = query_engine.query(
    "give me the summary of the project 41c2be0ed45342a687cad02d4d20b1cd, max 3 sentense",
)
display_response(response)

**`Final Response:`** The project 41c2be0ed45342a687cad02d4d20b1cd involves a two-phase design-build multiple award construction contract primarily for U.S. military construction projects in Guam and other areas under the cognizance of NAVFAC Pacific. It is a Department of Defense project with a focus on constructing various military facilities such as barracks, administrative buildings, and communication facilities. The procurement process includes evaluation factors like technical approach, experience, safety, and small business utilization for a contract anticipated to be 96 months with task orders ranging from $25,000,000 to $1,000,000,000.

In [40]:
query_engine = basic_index.as_query_engine(
    vector_store_query_mode="hybrid", similarity_top_k=2, alpha=0.6
)
response = query_engine.query(
    "Can you list plumbing projects with an offer due date in May 2024",
)
display_response(response)

**`Final Response:`** There are no plumbing projects with an offer due date in May 2024 based on the provided context information.

In [41]:
query_engine = basic_index.as_query_engine(
    vector_store_query_mode="hybrid", similarity_top_k=2, alpha=0.6
)
response = query_engine.query(
    "What is the size of the project 41c2be0ed45342a687cad02d4d20b1cd",
)
display_response(response)

**`Final Response:`** The size of project 41c2be0ed45342a687cad02d4d20b1cd is anticipated to be $15 billion for all contracts combined, with task orders ranging from $25,000,000 to $1,000,000,000.

In [44]:
# Favor BM25 (key word search)
query_engine = basic_index.as_query_engine(
    vector_store_query_mode="hybrid", similarity_top_k=2, alpha=0.6
)
response = query_engine.query(
    "what are all the projects that do not have a set-aside",
)
display_response(response)

**`Final Response:`** The projects that do not have a set-aside are as follows:
1. Project titled "J&A FOR INDEFINITE DELIVERY INDEFINITE QUANTITY ARCHITECT-ENGINEER SERVICES FOR DESIGN, ENGINEERING, SPECIFICATION WRITING, COST ESTIMATING, AND RELATED SERVICES AT VARIOUS LOCATIONS UNDER THE COGNIZANCE OF NAVAL FACILITIES ENGINEERING COMMAND, PACIFIC" under the Department of Defense.
2. Project titled "REPLACE PLUMBING AND FIRE PROTECTION SYSTEMS (B150), TIN CITY, ALASKA (TIN006)" under the Department of Defense.

In [45]:
# Favor BM25 (key word search)
query_engine = basic_index.as_query_engine(
    vector_store_query_mode="hybrid", similarity_top_k=2, alpha=0.6
)
response = query_engine.query(
    "I am interested in learning more about 41c2be0ed45342a687cad02d4d20b1cd. Can you provide a summary of the project’s scope of work?",
)
display_response(response)

**`Final Response:`** The project with ID 41c2be0ed45342a687cad02d4d20b1cd involves a design-build multiple award construction contract primarily for U.S. military construction projects in Guam and other areas under the cognizance of NAVFAC Pacific. The Department of Defense is responsible for this project, which includes construction services for various military facilities such as barracks, administrative buildings, communication facilities, and more. The procurement process is a two-phase design-build, best value, trade-off procurement with evaluation factors including technical approach, experience, performance confidence assessment, safety, and small business utilization. The contract is anticipated to be 96 months with a base period of 60 months and three 12-month option periods, with task orders ranging from $25,000,000 to $1,000,000,000.

union result with SQL index